For ‘K’ employees, we are given a list of intervals representing the working hours of each employee. Our goal is to find out if there is a free interval that is common to all employees. You can assume that each list of employee working hours is sorted on the start time. <br/>

Example 1: <br/>
Input: Employee Working Hours=[[[1,3], [5,6]], [[2,3], [6,8]]] <br/>
Output: [3,5] <br/>
Explanation: Both the employess are free between [3,5]. <br/>

Example 2: <br/>
Input: Employee Working Hours=[[[1,3], [9,12]], [[2,4]], [[6,8]]] <br/>
Output: [4,6], [8,9] <br/>
Explanation: All employess are free between [4,6] and [8,9]. <br/>

Example 3: <br/>
Input: Employee Working Hours=[[[1,3]], [[2,4]], [[3,5], [7,9]]] <br/>
Output: [5,7] <br/>
Explanation: All employess are free between [5,7]. 

# Two loops and Extra Space - O(N) runtime, O(N) space where N is the total number of intervals

In [11]:
class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end


def find_employee_free_time(schedule):
    if not schedule or len(schedule) == 0:
        return []
    
    result = []
    min_time, max_time = float('inf'), 0

    for i, s in enumerate(schedule):
        min_time = min(min_time, schedule[i][0].start)
        max_time = max(max_time, schedule[i][-1].end)

    time_arr = [1] * max_time

    for i, s in enumerate(schedule):
        for j, interval in enumerate(s):
            for k in range(interval.start, interval.end):
                time_arr[k] = 0


    ctr = min_time
    while ctr < max_time:
        if time_arr[ctr] == 1:
            min_bound = ctr
            while time_arr[ctr] == 1:
                ctr += 1
            max_bound = ctr
            result.append(Interval(min_bound, max_bound))

        ctr += 1

    return result

# Merge Interval - Heap - O(N * Log K) runtime, O(K) space where ‘N’ is the total number of intervals and ‘K’ is the total number of employees

In [12]:
from heapq import *


class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end

class EmployeeInterval:

    def __init__(self, interval, employeeIndex, intervalIndex):
        self.interval = interval  # interval representing employee's working hours
        # index of the list containing working hours of this employee
        self.employeeIndex = employeeIndex
        self.intervalIndex = intervalIndex  # index of the interval in the employee list

    def __lt__(self, other):
        # min heap based on meeting.end
        return self.interval.start < other.interval.start

def find_employee_free_time(schedule):
    if schedule is None:
        return []

    n = len(schedule)
    result, minHeap = [], []

    # insert the first interval of each employee to the queue
    for i in range(n):
        heappush(minHeap, EmployeeInterval(schedule[i][0], i, 0))

    previousInterval = minHeap[0].interval
    while minHeap:
        queueTop = heappop(minHeap)
        # if previousInterval is not overlapping with the next interval, insert a free interval
        if previousInterval.end < queueTop.interval.start:
            result.append(Interval(previousInterval.end,
                                   queueTop.interval.start))
            previousInterval = queueTop.interval
        else:  # overlapping intervals, update the previousInterval if needed
            if previousInterval.end < queueTop.interval.end:
                previousInterval = queueTop.interval

        # if there are more intervals available for the same employee, add their next interval
        employeeSchedule = schedule[queueTop.employeeIndex]
        if len(employeeSchedule) > queueTop.intervalIndex + 1:
            heappush(minHeap, EmployeeInterval(employeeSchedule[queueTop.intervalIndex + 1], queueTop.employeeIndex,
                                               queueTop.intervalIndex + 1))

    return result